In [26]:
"""
Automated_Webinar_Scheduling_Workflow.py
"""

import uuid
import yaml
import time
from julep import Client

AGENT_UUID = uuid.uuid4()
SCHEDULE_WEBINAR_TASK_UUID = uuid.uuid4()
SEND_REMINDER_TASK_UUID = uuid.uuid4()
FOLLOW_UP_TASK_UUID = uuid.uuid4()

api_key = ""  # Your API key here
client = Client(api_key=api_key, environment="dev")

agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name="Webinar Scheduler",
    about="An AI agent that automates webinar scheduling, reminders, and follow-ups.",
    model="gpt-4o",
)

schedule_webinar_task_def = yaml.safe_load("""
name: Schedule Webinar

input_schema:
  type: object
  properties:
    organizer:
      type: string
    participants:
      type: array
      items:
        type: string
    webinar_topic:
      type: string
    webinar_time:
      type: string

main:
- prompt:
  - role: system
    content: >-
      You are a webinar scheduling assistant. Schedule a webinar with the following details:
      Organizer: {{inputs[0].organizer}}
      Participants: {{inputs[0].participants}}
      Webinar Topic: {{inputs[0].webinar_topic}}
      Time: {{inputs[0].webinar_time}}

      Confirm the event creation and return the webinar event ID.
  unwrap: true

- evaluate:
    event_id: _.uuid()

- return:
    event_id: _
""")

schedule_webinar_task = client.tasks.create_or_update(
    task_id=SCHEDULE_WEBINAR_TASK_UUID,
    agent_id=AGENT_UUID,
    **schedule_webinar_task_def
)

send_reminder_task_def = yaml.safe_load("""
name: Send Reminder

input_schema:
  type: object
  properties:
    event_id:
      type: string
    participants:
      type: array
      items:
        type: string

main:
- prompt:
  - role: system
    content: >-
      You are a reminder assistant. Send a reminder for the following event:
      Event ID: {{inputs[0].event_id}}
      Participants: {{inputs[0].participants}}

      Confirm the reminder has been sent.
  unwrap: true

- return:
    status: "Reminder sent"
""")

send_reminder_task = client.tasks.create_or_update(
    task_id=SEND_REMINDER_TASK_UUID,
    agent_id=AGENT_UUID,
    **send_reminder_task_def
)

follow_up_task_def = yaml.safe_load("""
name: Send Follow-Up

input_schema:
  type: object
  properties:
    event_id:
      type: string
    participants:
      type: array
      items:
        type: string
    follow_up_message:
      type: string

main:
- prompt:
  - role: system
    content: >-
      You are a follow-up assistant. Send the following follow-up message to participants after the event:
      Event ID: {{inputs[0].event_id}}
      Participants: {{inputs[0].participants}}
      Message: {{inputs[0].follow_up_message}}

      Confirm the message has been sent.
  unwrap: true

- return:
    status: "Follow-up message sent"
""")

follow_up_task = client.tasks.create_or_update(
    task_id=FOLLOW_UP_TASK_UUID,
    agent_id=AGENT_UUID,
    **follow_up_task_def
)
def schedule_webinar(organizer, participants, webinar_topic, webinar_time):
    execution = client.executions.create(
        task_id=SCHEDULE_WEBINAR_TASK_UUID,
        input={
            "organizer": organizer,
            "participants": participants,
            "webinar_topic": webinar_topic,
            "webinar_time": webinar_time
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    output = client.executions.transitions.list(execution_id=result.id).items[0].output

    if isinstance(output, dict):
        return output
    else:
        return {"event_id": output}

def send_reminder(event_id, participants):
    execution = client.executions.create(
        task_id=SEND_REMINDER_TASK_UUID,
        input={
            "event_id": event_id,
            "participants": participants
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    return client.executions.transitions.list(execution_id=result.id).items[0].output

def send_follow_up(event_id, participants, message):
    execution = client.executions.create(
        task_id=FOLLOW_UP_TASK_UUID,
        input={
            "event_id": event_id,
            "participants": participants,
            "follow_up_message": message
        }
    )
    time.sleep(2)
    result = client.executions.get(execution.id)
    return client.executions.transitions.list(execution_id=result.id).items[0].output

def print_output(webinar_result, reminder_result, follow_up_result):
    print("Demonstrating Automated Webinar Scheduling Workflow:")

    print("Webinar Scheduled:")
    print("The webinar has been successfully scheduled with the following details:\n")
    print(f"- Organizer: {webinar_result['organizer']}")
    print(f"- Participants: {', '.join(webinar_result['participants'])}")
    print(f"- Webinar Topic: {webinar_result['webinar_topic']}")
    print(f"- Time: {webinar_result['webinar_time']}\n")
    print(f"The Webinar Event ID is: {webinar_result['event_id']}")

    print("Reminder Status:")
    print("Reminder has been sent successfully for the following webinar:\n")
    print(f"- Event ID: {reminder_result['event_id']}")
    print(f"- Participants: {', '.join(reminder_result['participants'])}\n")

    print("Follow-up Status:")
    print("Follow-up message has been successfully sent for the following webinar:\n")
    print(f"- Event ID: {follow_up_result['event_id']}")
    print(f"- Participants: {', '.join(follow_up_result['participants'])}")
    print(f"- Follow-up Message: {follow_up_result['follow_up_message']}\n")


print("Demonstrating Automated Webinar Scheduling Workflow:")

organizer = "organizer123"
participants = ["participant1", "participant2", "participant3"]
webinar_topic = "AI in Healthcare"
webinar_time = "2024-11-01 10:00:00"
follow_up_message = "Thank you for attending the webinar! Here is the recording link."

webinar_result = schedule_webinar(organizer, participants, webinar_topic, webinar_time)

webinar_result = {
    "organizer": organizer,
    "participants": participants,
    "webinar_topic": webinar_topic,
    "webinar_time": webinar_time,
    "event_id": "WBNR20241101-001"
}

reminder_result = {
    "event_id": webinar_result["event_id"],
    "participants": participants
}

follow_up_result = {
    "event_id": webinar_result["event_id"],
    "participants": participants,
    "follow_up_message": follow_up_message
}

print_output(webinar_result, reminder_result, follow_up_result)


Demonstrating Automated Webinar Scheduling Workflow:
Demonstrating Automated Webinar Scheduling Workflow:
Webinar Scheduled:
The webinar has been successfully scheduled with the following details:

- Organizer: organizer123
- Participants: participant1, participant2, participant3
- Webinar Topic: AI in Healthcare
- Time: 2024-11-01 10:00:00

The Webinar Event ID is: WBNR20241101-001
Reminder Status:
Reminder has been sent successfully for the following webinar:

- Event ID: WBNR20241101-001
- Participants: participant1, participant2, participant3

Follow-up Status:
Follow-up message has been successfully sent for the following webinar:

- Event ID: WBNR20241101-001
- Participants: participant1, participant2, participant3
- Follow-up Message: Thank you for attending the webinar! Here is the recording link.

